Elemental

In [ ]:
from random import randint

random_gene       = lambda                                     : randint(0, 1)
random_chromosome = lambda                   chromosome_lenght : [random_gene() for _ in range(chromosome_lenght)]
random_generation = lambda chromosome_count, chromosome_lenght : [random_chromosome(chromosome_lenght)
                                                                  for _ in range(chromosome_count)]

Selection algorithms

In [ ]:
from random import random, uniform
from  numpy import float64

def rank(generation : [[0 or 1]], function=None) -> [0 or 1]: # parent
    parent : [0 or 1]
    
    p = 0
    r = random()
    
    N = len(generation)
    a = uniform(1, 2)
    b = 2 - a
    
    for i in range(N):
        p += 1/N * (a - (a-b) * (i-1)/(1 if N-1 == 0 else N-1))
        
        if p >= r:
            parent = generation[i]
            return parent

In [ ]:
from random import random
from  numpy import float64   

def roulette(generation : [[0 or 1]],          # parent
               function : lambda x: float64) -> [0 or 1]:
    
    parent : [0 or 1]
    
    p = 0
    r = random()
    
    s = sum(map(function, generation))
    
    # if s == 0:
    #     parent = generation[0]
    #     return parent
    
    for i in range(len(generation)):
        p += function(generation[i]) / s
        
        if p >= r:
            parent = generation[i]
            return parent
        
    parent = generation[i]     
    return parent

In [ ]:
from random import choice
from  numpy import float64          

def tournament(generation : [[0 or 1]],          # parent
                 function : lambda x: float64) -> [0 or 1]:
    
    mom = choice(generation)
    dad = choice(generation)
    
    parent             : [0 or 1] = min(mom, dad, key=function)
    return parent

Mate algorithms

In [ ]:
from random import random

def steady(mom: [0 or 1], dad: [0 or 1]) -> [0 or 1]:
    
    child = []
    for mom_gen, dad_gen in zip(mom, dad):
        if random() > 1/2: child += [mom_gen]
        else             : child += [dad_gen]
        
    return child
    
def one_point(mom: [0 or 1], dad: [0 or 1]) -> [0 or 1]: 
    point = randint(0, len(mom)-1)
    child = mom[:point] + dad[point:]
    return child
    
    
def two_points(mom: [0 or 1], dad: [0 or 1]) -> [0 or 1]:
    point1 = randint(0, len(mom)-2)
    point2 = randint(point1, len(mom)-1)
    child  = mom[:point1] + dad[point1:point2] + mom[point2:]
    return child

Mutation

In [ ]:
from random import random

def mutation(chromosome: [0 or 1]) -> [0 or 1]:
    p = 1/len(chromosome)
    
    for i in range(len(chromosome)):
        if random() < p:
            chromosome[i] = abs(chromosome[i] - 1)
            
    return chromosome

Converting a function to accept a chromosome argument (hmmmmmm)



In [ ]:
from numpy import float64

def function_conversion(function          : lambda x: float,
                        interval_min      : float,
                        interval_max      : float,
                        chromosome_length : float) -> lambda x: float64:
    
    return lambda x: float64(function(float64( interval_min + int(''.join(map(str, x)), 2) * (interval_max - interval_min) / 2**chromosome_length )))

Batch

In [ ]:
from numpy import float64
from copy import copy
from math import ceil

def batch(         function : lambda x: float64,
                     answer : float,
                   accuracy : float,
                epoch_count : int,
           chromosome_count : int, # return (successfulBatch_flag, successfulEpoch_number, leader)
          chromosome_lenght : int,
                  selection : rank or roulette or tournament,
                       mate : steady or one_point or two_points) -> (bool, int, [0 or 1]):
    
    successfulBatch_flag   :     bool = False
    successfulEpoch_number :      int = None
    leader                 : [0 or 1] = None
    
    generation         : [[0 or 1]] = random_generation(chromosome_count, chromosome_lenght)
    generation         : [[0 or 1]] = sorted(generation, key=function)
    
    childCount_percent : int = 90 # CONST
    childCount         : int = ceil(len(generation)*(childCount_percent/100))
    
    generation         : [[0 or 1]] = generation[:-childCount]
    virtual_generation : [[0 or 1]] = copy(generation)
    leader             :   [0 or 1] = generation[0]
    
    for epoch_number in range(epoch_count):
          for _ in range(childCount):
                mom = selection(virtual_generation, function)
                
                if len(virtual_generation) != 1:
                  virtual_generation.remove(mom)
                  
                dad = selection(virtual_generation, function)
                virtual_generation.remove(dad)
                child = mutation(mate(mom, dad))
                
                generation += [child]
                virtual_generation += [mom, dad]
                virtual_generation.sort(key=function)
                
          generation         = sorted(generation, key=function)
          leader             = generation[0]
          
          if abs(function(leader) - answer) <= accuracy:
                successfulBatch_flag   = True
                successfulEpoch_number = epoch_number
                leader                 = leader
                
                return successfulBatch_flag, successfulEpoch_number, leader
          
          generation         = generation[:-childCount]
          virtual_generation = copy(generation)
          
    else:
          return successfulBatch_flag, successfulEpoch_number, leader

Main move



In [ ]:
from numpy import mean, float64
from math  import ceil, log2

def fit(         function : lambda x: float,
                 interval : (float, float),
                   answer : float,
                 accuracy : float,
                selection : rank or roulette or tournament,
                     mate : steady or one_point or two_points,
         chromosome_count : int,
              epoch_count : int,     # return (reliability, expenses, leaders)
              batch_count : int)    -> (float, float, list):
  
  reliability : float( 1 - int(failedBatch_count) / int(batch_count) )
  expenses    : float( mean(list(successfulEpoch_numbers)) )
  leaders     :  list( sorted(list(leaders_unsort), key=function) )
  
  failedBatch_count       : int        = batch_count
  successfulEpoch_numbers : list       = []
  leaders_unsort          : [[0 or 1]] = []
  
  # parameters conversion
  interval_min, interval_max       = interval[0], interval[1]
  chromosome_length                = ceil(log2(interval_max - interval_min) - log2(accuracy))
  function:      lambda x: float64 = function_conversion(function,
                                                         interval_min,interval_max,
                                                         chromosome_length)
  
  for batch_number in range(batch_count):
    successfulBatch_flag, successfulEpoch_number, leader = batch(function,
                                                                 answer,
                                                                 accuracy,
                                                                 epoch_count,
                                                                 chromosome_count,
                                                                 chromosome_length,
                                                                 selection,
                                                                 mate)
    
    if successfulBatch_flag:
      failedBatch_count       -= 1
      successfulEpoch_numbers += [successfulEpoch_number]
      leaders_unsort          += [leader]
      
  reliability = 1 - failedBatch_count / batch_count
  expenses    = mean(successfulEpoch_numbers)
  leaders     = sorted(leaders_unsort, key=function)
  
  return {'надёжность' : reliability,
             'затраты' : expenses,
              'лидеры' : leaders}

Running one test function



In [ ]:
from numpy import seterr
seterr(divide='ignore')

objective_function_1 = lambda x: x**2                     # answer = 0,       interval = (-16, 16)
objective_function_2 = lambda x: (x**2 + 3*x - 9)/3       # answer = -3.75,   interval = (-18, 14)
objective_function_3 = lambda x: (2**(x**3))/(8*x)        # answer = 0.223,   interval = (0.01, 32)
objective_function_4 = lambda x: (x**2 + 8)/2 - 2*x       # answer = 2,       interval = (-14, 18)
objective_function_5 = lambda x: (2*x**4 - 8*x - 5)/(5/4) # answer = -8.8,    interval = (-15, 17)
                                                          # answer = -11.735, interval = (-4, 4)
objective_function_6 = lambda x: ((16**(x**2)) + (18*(x**3)) - (16*x)) / (10*(x**2)) - (10*(x**2))

fit(        function=objective_function_3,
                            interval=(-5, 5),
                             answer=-11.735,
                            accuracy=0.001,
                           selection=roulette, # rank, roulette, tournament
                                mate=one_point, # steady, one_point, two_points
                    chromosome_count=100,
                         epoch_count=30,
                         batch_count=50)

Testing: is the leading chromosome true to the global minimum?



In [ ]:
function_conversion(objective_function_6, -4, 4, 13)([0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0])